In [9]:
from selenium import webdriver
from selenium.common.exceptions import WebDriverException, ElementClickInterceptedException, NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
import re
import time
import pandas as pd

In [10]:
servico = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=servico)
driver.maximize_window()
driver.get(r'https://www.vivareal.com.br/venda/rj/rio-de-janeiro/#area-desde=70&onde=Brasil,Rio%20de%20Janeiro,Rio%20de%20Janeiro,,,,,,BR%3ERio%20de%20Janeiro%3ENULL%3ERio%20de%20Janeiro,,,;,Rio%20de%20Janeiro,Rio%20de%20Janeiro,Zona%20Sul,Flamengo,,,neighborhood,BR%3ERio%20de%20Janeiro%3ENULL%3ERio%20de%20Janeiro%3EZona%20Sul%3EFlamengo,,,;,Rio%20de%20Janeiro,Rio%20de%20Janeiro,Zona%20Sul,Botafogo,,,neighborhood,BR%3ERio%20de%20Janeiro%3ENULL%3ERio%20de%20Janeiro%3EZona%20Sul%3EBotafogo,,,;,Rio%20de%20Janeiro,Rio%20de%20Janeiro,Zona%20Sul,Copacabana,,,neighborhood,BR%3ERio%20de%20Janeiro%3ENULL%3ERio%20de%20Janeiro%3EZona%20Sul%3ECopacabana,,,;,Rio%20de%20Janeiro,Rio%20de%20Janeiro,Zona%20Sul,Humait%C3%A1,,,neighborhood,BR%3ERio%20de%20Janeiro%3ENULL%3ERio%20de%20Janeiro%3EZona%20Sul%3EHumaita,,,;,Rio%20de%20Janeiro,Rio%20de%20Janeiro,Zona%20Sul,Ipanema,,,neighborhood,BR%3ERio%20de%20Janeiro%3ENULL%3ERio%20de%20Janeiro%3EZona%20Sul%3EIpanema,,,;,Rio%20de%20Janeiro,Rio%20de%20Janeiro,Zona%20Sul,Lagoa,,,neighborhood,BR%3ERio%20de%20Janeiro%3ENULL%3ERio%20de%20Janeiro%3EZona%20Sul%3ELagoa,,,;,Rio%20de%20Janeiro,Rio%20de%20Janeiro,Zona%20Sul,Jardim%20Bot%C3%A2nico,,,neighborhood,BR%3ERio%20de%20Janeiro%3ENULL%3ERio%20de%20Janeiro%3EZona%20Sul%3EJardim%20Botanico,,,&preco-ate=1300000')
time.sleep(5)

# Handling cookies acception (Lidando com a aceitação de cookies)
WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.XPATH,'//*[@id="cookie-notifier-cta"]'))).click()
time.sleep(5/2)

[WDM] - Downloading: 100%|██████████| 6.30M/6.30M [00:00<00:00, 7.66MB/s]


In [8]:
! pip install pandas

  Using cached pandas-2.0.3-cp310-cp310-win_amd64.whl (10.7 MB)
  Using cached pytz-2023.3-py2.py3-none-any.whl (502 kB)
  Using cached tzdata-2023.3-py2.py3-none-any.whl (341 kB)
  Using cached numpy-1.25.1-cp310-cp310-win_amd64.whl (15.0 MB)


In [11]:
soup = BeautifulSoup(driver.page_source, 'html.parser')

In [25]:
div_list = soup.find_all('div', {'class':'property-card__content'})
div_list_link = soup.find_all('a', {'class':'property-card__content-link js-card-title'})
div_list_link[0]['href']

'/imovel/apartamento-2-quartos-copacabana-zona-sul-rio-de-janeiro-com-garagem-85m2-venda-RS850000-id-2643792930/'

In [28]:
for d,l in zip(div_list, div_list_link):
  
    print("Link", "{}{}".format('https://www.vivareal.com.br', l.get('href')))
    break

Link https://www.vivareal.com.br/imovel/apartamento-2-quartos-copacabana-zona-sul-rio-de-janeiro-com-garagem-85m2-venda-RS850000-id-2643792930/


In [29]:
class ScraperVivaReal:
    wait_time = 5

    def __init__(self, url):
        # Initializing the webdriver (Inicializando o webdriver)

        servico = Service(ChromeDriverManager().install())
        self.driver = webdriver.Chrome(service=servico)
        self.driver.maximize_window()
        self.driver.get(url)
        time.sleep(self.wait_time)

        # Handling cookies acception (Lidando com a aceitação de cookies)
        WebDriverWait(self.driver, self.wait_time).until(EC.element_to_be_clickable((By.XPATH,'//*[@id="cookie-notifier-cta"]'))).click()
        time.sleep(self.wait_time/2)

    def __scrape_page__(self):
        result = []

        # Extracting data from the page (Extraindo dados da página)
        try:
            soup = BeautifulSoup(self.driver.page_source, 'html.parser')
        except WebDriverException:
            print('Webdriver was manually quit by the user!') # I configure this exception before adding the option -headless to webdriver
            # Eu configuro esta exceção antes de adicionar a opção -headless ao webdriver
            return result

        # Finding property cards containing search results
        # Encontrar cartões de propriedade contendo resultados de pesquisa
        div_list = soup.find_all('div', {'class':'property-card__content'})
        div_list_link = soup.find_all('a', {'class':'property-card__content-link js-card-title'})

        # Iterating each card
        # Iterando cada cartão
        for d,l in zip(div_list, div_list_link):

            # Extracting info from card
            # Extraindo informações do cartão
            title = d.find('span', {'class': 'property-card__title js-cardLink js-card-title'}).get_text().strip()
            complete_address = d.find('span', {'class': 'property-card__address'}).get_text().strip()
            area = d.find('span', {'class': 'property-card__detail-value js-property-card-value property-card__detail-area js-property-card-detail-area'}).get_text().strip()
            rooms = d.find('li', {'class': 'property-card__detail-item property-card__detail-room js-property-detail-rooms'}).find('span', {'class': 'property-card__detail-value js-property-card-value'}).get_text().strip()
            baths = d.find('li', {'class': 'property-card__detail-item property-card__detail-bathroom js-property-detail-bathroom'}).find('span', {'class': 'property-card__detail-value js-property-card-value'}).get_text().strip()
            garage = d.find('li', {'class': 'property-card__detail-item property-card__detail-garage js-property-detail-garages'}).find('span', {'class': 'property-card__detail-value js-property-card-value'}).get_text().strip()
            url = "{}{}".format('https://www.vivareal.com.br', l.get('href'))
            # Extracting the price
            # Extraindo o preço
            try:
                price = d.find('div', {'class':'property-card__price js-property-card-prices js-property-card__price-small'}).find('p').get_text().strip()
            except AttributeError:
                price = "N/I"

            # Splitting the address
            # Dividindo o endereço
            add_list = re.split(',|-', complete_address)
            add_list = [ item.strip() for item in add_list ]
            if len(add_list) == 2:
                city, st = add_list
                neibhood = 'N/I'
                address = 'N/I'
                number = 'N/I'
            if len(add_list) == 3:
                neibhood, city, st = add_list
                address = 'N/I'
                number = 'N/I'
            if len(add_list) == 4:
                address, neibhood, city, st = add_list
                number = 'N/I'
            elif len(add_list) == 5:
                address, number, neibhood, city, st = add_list

            # Adding the result into a dicionary and appending the dict to a result list
            # Adicionar o resultado a um dicionário e anexar o dict a uma lista de resultados
            row = { 'Título': title, 'Endereço': address, 'Número': number, 'Bairro': neibhood, 'Cidade': city, 'Estado': st, 'Área': area, 'Quartos': rooms, 'Banheiros': baths, 'Vagas': garage, 'Preço': price , 'URL': url}
            result.append(row)
        return result

    def __next_page__(self):
        # Finding the "Next Page" button element
        # Encontrando o elemento do botão "Próxima página"
        next_element = self.driver.find_element('xpath', f'//*[@id="js-site-main"]/div[2]/div[1]/section/div[2]/div[2]/div/ul/li[9]/button')

        try:
            # Trying to click it (Tentando clicar nele)
            next_element.click()
            time.sleep(self.wait_time)
            return True
        # Treating some exceptions (element not found and element not clickable)
        # Tratando algumas exceções (elemento não encontrado e elemento não clicável)
        except ElementClickInterceptedException:
            print('"Próxima Página" element is not clickable!')
        except NoSuchElementException:
            print('"Próxima Página" element not found!')
        return False

    def run(self, output):
        has_next = True
        final_result = []
        # Getting the information!
        # Obtendo as informações!
        while has_next:
            results = self.__scrape_page__()
            final_result.extend(results)
            print('Got {} results! Total Found: {}'.format(len(results), len(final_result)))
            if len(results) == 0:
                break
            has_next = self.__next_page__()
        # Quitting Chrome
        # Saindo do Chrome
        self.driver.quit()
        # Exporting results to CSV
        # Exportando resultados para CSV
        df = pd.DataFrame(final_result)
        df.to_csv(output, sep=',')

S = ScraperVivaReal('https://www.vivareal.com.br/venda/rj/rio-de-janeiro/#area-desde=70&onde=Brasil,Rio%20de%20Janeiro,Rio%20de%20Janeiro,,,,,,BR%3ERio%20de%20Janeiro%3ENULL%3ERio%20de%20Janeiro,,,;,Rio%20de%20Janeiro,Rio%20de%20Janeiro,Zona%20Sul,Flamengo,,,neighborhood,BR%3ERio%20de%20Janeiro%3ENULL%3ERio%20de%20Janeiro%3EZona%20Sul%3EFlamengo,,,;,Rio%20de%20Janeiro,Rio%20de%20Janeiro,Zona%20Sul,Botafogo,,,neighborhood,BR%3ERio%20de%20Janeiro%3ENULL%3ERio%20de%20Janeiro%3EZona%20Sul%3EBotafogo,,,;,Rio%20de%20Janeiro,Rio%20de%20Janeiro,Zona%20Sul,Copacabana,,,neighborhood,BR%3ERio%20de%20Janeiro%3ENULL%3ERio%20de%20Janeiro%3EZona%20Sul%3ECopacabana,,,;,Rio%20de%20Janeiro,Rio%20de%20Janeiro,Zona%20Sul,Humait%C3%A1,,,neighborhood,BR%3ERio%20de%20Janeiro%3ENULL%3ERio%20de%20Janeiro%3EZona%20Sul%3EHumaita,,,;,Rio%20de%20Janeiro,Rio%20de%20Janeiro,Zona%20Sul,Ipanema,,,neighborhood,BR%3ERio%20de%20Janeiro%3ENULL%3ERio%20de%20Janeiro%3EZona%20Sul%3EIpanema,,,;,Rio%20de%20Janeiro,Rio%20de%20Janeiro,Zona%20Sul,Lagoa,,,neighborhood,BR%3ERio%20de%20Janeiro%3ENULL%3ERio%20de%20Janeiro%3EZona%20Sul%3ELagoa,,,;,Rio%20de%20Janeiro,Rio%20de%20Janeiro,Zona%20Sul,Jardim%20Bot%C3%A2nico,,,neighborhood,BR%3ERio%20de%20Janeiro%3ENULL%3ERio%20de%20Janeiro%3EZona%20Sul%3EJardim%20Botanico,,,&preco-ate=1300000')
S.run('output.csv')

Got 36 results! Total Found: 36
Got 36 results! Total Found: 72
Got 36 results! Total Found: 108
Got 36 results! Total Found: 144
Got 36 results! Total Found: 180
Got 36 results! Total Found: 216
Got 36 results! Total Found: 252
Got 36 results! Total Found: 288
Got 36 results! Total Found: 324
Got 36 results! Total Found: 360
Got 36 results! Total Found: 396
Got 36 results! Total Found: 432
Got 36 results! Total Found: 468
Got 36 results! Total Found: 504
Got 36 results! Total Found: 540
Got 36 results! Total Found: 576
Got 36 results! Total Found: 612
Got 36 results! Total Found: 648
Got 36 results! Total Found: 684
Got 36 results! Total Found: 720
Got 36 results! Total Found: 756
Got 36 results! Total Found: 792
Got 36 results! Total Found: 828
Got 36 results! Total Found: 864
Got 36 results! Total Found: 900
Got 36 results! Total Found: 936
Got 36 results! Total Found: 972
Got 36 results! Total Found: 1008
Got 36 results! Total Found: 1044
Got 36 results! Total Found: 1080
Got 36 re

In [5]:
df = pd.read_csv('output.csv')

In [7]:
df.to_excel('output.xlsx', index=False)